In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.layers import LSTM 
from keras.layers import Embedding, Dense, Dropout
from keras.layers import LeakyReLU

# Function to perform pre-processing steps

At first tokenization will be performed then stemming will be performed over tokens

In [7]:
def token_stems(text):
        
    tokens=tokenizing(text) 
    stems=stemming(tokens)
    return stems       

# Stemming Function

In [8]:
from nltk.stem.snowball import SnowballStemmer
stemmer= SnowballStemmer("english")


def stemming(text):
    
    stems =[stemmer.stem(t) for t in text]
    return stems

# Tokenization Function

In [9]:
def tokenizing(text):
    
    #breaking each word and making them tokens
    tokens=[word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    #storing only alpha tokens
    filtered_tokens=[]
    for token in tokens:
        if (re.search('[a-zA-Z]|\'', token)):
            filtered_tokens.append(token)

    return filtered_tokens

In [10]:
train= pd.read_excel("D:\\virtual\\leave_final.xlsx")
tra
#train.shape

In [ ]:
docs= train['Data']

In [ ]:
tokens = []
for i in docs:
    temp = token_stems(i)
    tokens.append(temp)


In [ ]:
#print(docs[1],"\n\n",tokens[1])

In [33]:
x, y = np.asarray(tokens) , np.asarray(train['Kind'])

In [34]:
le = LabelEncoder()
y = le.fit_transform(y)
y= to_categorical(y)
#y[:10]

In [35]:
xtrain, xtest, ytrain, ytest= train_test_split(x, y, test_size= 0.3, random_state=100)

In [36]:
max_words = 20000
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(x)

In [37]:
sequences = tok.texts_to_sequences(x)
test_sequences = tok.texts_to_sequences(xtest)

In [38]:
max_len =200

sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)
#test_sequences_matrix[1]

# Model 

In [41]:
def model():
    model = Sequential()
    model.add(Embedding(max_words,50,input_length=max_len))
    model.add(LSTM(50, activation='tanh',dropout=0.1))
    #model.add(Dense(100, activation='relu'))
    #model.add(Dropout(0.01))
    #model.add(Dense(20, activation='relu'))
    
    model.add(Dropout(0.01))    
    
    
    print(model.add(Dense(4, activation='softmax')))
    
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['mae','acc'])
    return model

In [42]:
c= model()

c.fit(sequences_matrix,y,epochs=24, validation_split=0.2)

None
Train on 397 samples, validate on 100 samples
Epoch 1/24
397/397 [==============================] - 3s 8ms/step - loss: 0.5529 - mean_absolute_error: 0.3713 - acc: 0.7500 - val_loss: 0.5858 - val_mean_absolute_error: 0.3832 - val_acc: 0.7500
Epoch 2/24
397/397 [==============================] - 2s 4ms/step - loss: 0.4999 - mean_absolute_error: 0.3321 - acc: 0.7538 - val_loss: 0.9195 - val_mean_absolute_error: 0.4543 - val_acc: 0.5000
Epoch 3/24
397/397 [==============================] - 2s 4ms/step - loss: 0.4799 - mean_absolute_error: 0.3144 - acc: 0.7513 - val_loss: 0.7011 - val_mean_absolute_error: 0.4149 - val_acc: 0.7500
Epoch 4/24
397/397 [==============================] - 2s 4ms/step - loss: 0.4599 - mean_absolute_error: 0.3095 - acc: 0.7815 - val_loss: 0.7384 - val_mean_absolute_error: 0.4220 - val_acc: 0.5700
Epoch 5/24
397/397 [==============================] - 2s 4ms/step - loss: 0.4227 - mean_absolute_error: 0.2967 - acc: 0.8394 - val_loss: 0.7540 - val_mean_absolute_e

In [43]:
"""from keras.layers import Dense
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit


m = model()
# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=model, epochs=15, verbose=2)
kfold = ShuffleSplit(n_splits=15, test_size=0.3, random_state=100)
m.fit(cross_val_score(estimator,sequences_matrix,y, cv=kfold))"""


'from keras.layers import Dense\nfrom keras.models import Sequential\nfrom keras.wrappers.scikit_learn import KerasClassifier\nfrom sklearn.model_selection import StratifiedKFold\nfrom sklearn.model_selection import cross_val_score\nfrom sklearn.model_selection import ShuffleSplit\n\n\nm = model()\n# evaluate model with standardized dataset\nestimator = KerasClassifier(build_fn=model, epochs=15, verbose=2)\nkfold = ShuffleSplit(n_splits=15, test_size=0.3, random_state=100)\nm.fit(cross_val_score(estimator,sequences_matrix,y, cv=kfold))'

In [44]:
accuracy = c.evaluate(test_sequences_matrix,ytest)

150/150 [==============================] - 0s 811us/step


In [45]:
#save your model
c.save('test.h5')